<a href="https://colab.research.google.com/github/dp22acn/Data_Science_Project/blob/main/Python_Code_2_2_Automated__approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy

In [ ]:
!pip install pyvis

In [ ]:
import spacy
import pandas as pd
import networkx as nx
import numpy as np
import re
import os
from nltk.tokenize import sent_tokenize
from spacy import displacy
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Add the sentencizer to the pipeline
nlp.add_pipe("sentencizer")

First Version OF creating NER MOdel

In [ ]:
all_books = [b for b in os.scandir("/content/drive/MyDrive/mahatxt") if b.is_file()]

In [ ]:
all_books

In [ ]:
book = all_books[0]
book_text = open(book).read()
# Increase the max_length limit
nlp.max_length = len(book_text) + 1000  # Adding a buffer to be safe
book_doc = nlp(book_text)


In [ ]:
displacy.render(book_doc[0:500], style="ent", jupyter=True)

In [ ]:
character_df = pd.read_csv('/content/Character_Nmaes.csv')

In [ ]:
sent_entity_df = []
for sent in book_doc.sents:
    entity_list = [ent.text for ent in sent.ents]
    sent_entity_df.append({"sentence":sent, "entities":entity_list})

sent_entity_df = pd.DataFrame(sent_entity_df)

sent_entity_df.head()

In [ ]:
def filter_entity(entity_list, character_df):
  return[ent for ent in entity_list
         if ent in list(character_df.character)]

In [ ]:
sent_entity_df['character_entities'] = sent_entity_df['entities'].apply(lambda x: (filter_entity(x, character_df)))
sent_entity_df_fil = sent_entity_df[sent_entity_df['character_entities'].map(len) >0]

sent_entity_df_fil.head(10)

In [ ]:
pd.reset_option('^display.', silent=True)
sent_entity_df_fil

In [ ]:
window_size_ = 7
relations = []


for i in range(len(sent_entity_df_fil)): # Iterate over the length of the DataFrame
  end_i = min(i + window_size_, len(sent_entity_df_fil) -1 )  # Ensure end_i is within bounds
  # Extract character entities and flatten the list
  char_list = [item for sublist in sent_entity_df_fil.loc[i:end_i, 'character_entities'].tolist() for item in sublist]

  char_unq = [char_list[i] for i in range(len(char_list)) if (i == 0) or char_list[i] != char_list[i - 1]]


  if len(char_unq) > 1:
    for idx, a in enumerate(char_unq[:-1]):
      for b in char_unq[idx + 1:]:
        relations.append({'Source': a, 'Target': b})


In [ ]:
relations_df = pd.DataFrame(relations)
relations_df['value'] = 1
# Sort only the 'Source' and 'Target' columns before grouping
relations_df = relations_df.sort_values(by=['Source', 'Target'])
relations_df = relations_df.groupby(['Source', 'Target'], sort=False, as_index=False).sum()
relations_df.head()

In [ ]:
pd.reset_option('^display.', silent=True)
relations_df

In [ ]:
G = nx.from_pandas_edgelist(relations_df, source="Source", target="Target", edge_attr="value", create_using=nx.DiGraph())

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

pos = nx.kamada_kawai_layout(G)

nx.draw(G, node_color='green', with_labels=True, edge_cmap=plt.cm.Blues, pos=pos)
plt.figure(figsize=(12, 12))
plt.show()

In [ ]:
from pyvis.network import Network
net = Network(notebook=True, width="1000px", height="1000px", )

node_degree = dict(G.degree)
nx.set_node_attributes(G, node_degree, 'size')

net.from_nx(G)
net.show("graph.html")


# Exporting  to GEXF
nx.write_gexf(G, "graph_book_1.gexf")

Converted the model into Def funttion to go through the loop for all books

In [ ]:
import spacy
import pandas as pd
import numpy as np
import networkx as nx
from nltk.tokenize import sent_tokenize
from pyvis.network import Network
import os
import matplotlib.pyplot as plt

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Add sentencizer to the pipeline
nlp.add_pipe("sentencizer", last=True)

# Function to load a book and process it
def process_book(book_path, character_df, window_size=10):
    with open(book_path, "r", encoding="utf-8") as file:
        book_text = file.read()

    # Adjust spaCy max_length
    nlp.max_length = len(book_text) + 1000

    # Process the book text
    book_doc = nlp(book_text)

    # Extract sentences and entities
    sent_entity_df = []
    for sent in book_doc.sents:
        entity_list = [ent.text for ent in sent.ents]
        sent_entity_df.append({"sentence": sent, "entities": entity_list})

    sent_entity_df = pd.DataFrame(sent_entity_df)

    # Filter entities based on the character list
    sent_entity_df['character_entities'] = sent_entity_df['entities'].apply(
        lambda x: [ent for ent in x if ent in list(character_df.character)]
    )
    sent_entity_df_fil = sent_entity_df[sent_entity_df['character_entities'].map(len) > 0]

    # Extract relationships
    relations = []
    for i in range(len(sent_entity_df_fil)):
        end_i = min(i + window_size, len(sent_entity_df_fil) - 1)
        char_list = [
            item for sublist in sent_entity_df_fil.loc[i:end_i, 'character_entities'].tolist() for item in sublist
        ]
        char_unq = [char_list[i] for i in range(len(char_list)) if (i == 0) or char_list[i] != char_list[i - 1]]
        if len(char_unq) > 1:
            for idx, a in enumerate(char_unq[:-1]):
                for b in char_unq[idx + 1:]:
                    relations.append({'Source': a, 'Target': b})

    # Create a DataFrame for relationships
    relations_df = pd.DataFrame(relations)
    if not relations_df.empty:
        relations_df = pd.DataFrame(np.sort(relations_df.values, axis=1), columns=relations_df.columns)
        relations_df["value"] = 1
        relations_df = relations_df.groupby(["Source", "Target"], sort=False, as_index=False).sum()
    return relations_df

# Function to create and save the graph
def create_and_save_graph(relations_df, book_number):
    # Create the network graph
    G = nx.from_pandas_edgelist(relations_df, source="Source", target="Target", edge_attr="value", create_using=nx.DiGraph())

    # Visualize using matplotlib
    pos = nx.kamada_kawai_layout(G)
    plt.figure(figsize=(12, 12))
    nx.draw(G, pos, node_color='green', with_labels=True, edge_cmap=plt.cm.Blues)
    plt.show()

    # Visualize using pyvis
    net = Network(notebook=True, width="1000px", height="1000px")
    node_degree = dict(G.degree)
    nx.set_node_attributes(G, node_degree, 'size')
    net.from_nx(G)
    net.show(f"graph_{book_number}.html")

    # Save as GEXF file
    nx.write_gexf(G, f"graph_book_{book_number}.gexf")
    print(f"GEXF file saved for Book {book_number} as graph_book_{book_number}.gexf")

# Main processing function for all books
def process_all_books(folder_path, character_csv, window_size=10):
    character_df = pd.read_csv(character_csv)
    all_books = [b for b in os.scandir(folder_path) if b.is_file() and b.name.endswith(".txt")]

    for idx, book in enumerate(all_books, start=1):
        print(f"Processing Book {idx}: {book.name}")
        relations_df = process_book(book.path, character_df, window_size)
        if not relations_df.empty:
            create_and_save_graph(relations_df, idx)
        else:
            print(f"No relationships found in Book {idx}: {book.name}")

# Run the process for all books in the folder
books_folder = "/content/drive/MyDrive/data"
character_csv = "/content/filtered_characters.csv"
process_all_books(books_folder, character_csv)


Version 2

In [ ]:
import os
import spacy
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Add sentencizer to the pipeline
nlp.add_pipe("sentencizer", last=True)

# Function to load a book and process it
def process_book(book_path, character_df, window_size=5):
    with open(book_path, "r", encoding="utf-8") as file:
        book_text = file.read()

    # Adjust spaCy max_length
    nlp.max_length = len(book_text) + 1000

    # Process the book text
    book_doc = nlp(book_text)

    # Extract sentences and entities
    sent_entity_df = []
    for sent in book_doc.sents:
        entity_list = [ent.text for ent in sent.ents]
        sent_entity_df.append({"sentence": sent, "entities": entity_list})

    sent_entity_df = pd.DataFrame(sent_entity_df)

    # Filter entities based on the character list
    sent_entity_df['character_entities'] = sent_entity_df['entities'].apply(
        lambda x: [ent for ent in x if ent in list(character_df.character)]
    )
    sent_entity_df_fil = sent_entity_df[sent_entity_df['character_entities'].map(len) > 0]

    # Extract relationships
    relations = []
    for i in range(len(sent_entity_df_fil)):
        end_i = min(i + window_size, len(sent_entity_df_fil) - 1)
        char_list = [
            item for sublist in sent_entity_df_fil.loc[i:end_i, 'character_entities'].tolist() for item in sublist
        ]
        char_unq = [char_list[i] for i in range(len(char_list)) if (i == 0) or char_list[i] != char_list[i - 1]]
        if len(char_unq) > 1:
            for idx, a in enumerate(char_unq[:-1]):
                for b in char_unq[idx + 1:]:
                    if a != b:  # Remove self-loops
                        relations.append({'Source': a, 'Target': b})

    # Create a DataFrame for relationships
    relations_df = pd.DataFrame(relations)
    if not relations_df.empty:
        relations_df["value"] = 1
        relations_df = relations_df.groupby(["Source", "Target"], sort=False, as_index=False).sum()
    return relations_df

# Function to create and save the graph
def create_and_save_graph(relations_df, book_number):
    # Create the directed graph
    G = nx.from_pandas_edgelist(
        relations_df, source="Source", target="Target", edge_attr="value", create_using=nx.DiGraph()
    )

    # Visualize using matplotlib
    pos = nx.kamada_kawai_layout(G)
    plt.figure(figsize=(12, 12))
    nx.draw(G, pos, node_color='green', with_labels=True, edge_cmap=plt.cm.Blues)
    plt.show()

    # Visualize using pyvis
    net = Network(notebook=True, width="1000px", height="1000px", directed=True)
    node_degree = dict(G.degree)
    nx.set_node_attributes(G, node_degree, 'size')
    net.from_nx(G)
    net.show(f"graph_{book_number}.html")

    # Save as GEXF file
    nx.write_gexf(G, f"graph_book_{book_number}.gexf")
    print(f"GEXF file saved for Book {book_number} as graph_book_{book_number}.gexf")

# Main processing function for all books
def process_all_books(folder_path, character_csv, window_size=10):
    character_df = pd.read_csv(character_csv)
    all_books = [b for b in os.scandir(folder_path) if b.is_file() and b.name.endswith(".txt")]

    for idx, book in enumerate(all_books, start=1):
        print(f"Processing Book {idx}: {book.name}")
        relations_df = process_book(book.path, character_df, window_size)
        if not relations_df.empty:
            create_and_save_graph(relations_df, idx)
        else:
            print(f"No relationships found in Book {idx}: {book.name}")

# Run the process for all books in the folder
books_folder = "/content/drive/MyDrive/mahatxt"
character_csv = "/content/Character_Nmaes.csv"
process_all_books(books_folder, character_csv)


Final Version

In [ ]:
import os
import spacy
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Add sentencizer to the pipeline
nlp.add_pipe("sentencizer", last=True)

# Function to load a book and process it
def process_book(book_path, character_df, alt_name_map, window_size=5):
    with open(book_path, "r", encoding="utf-8") as file:
        book_text = file.read()

    # Adjust spaCy max_length
    nlp.max_length = len(book_text) + 1000

    # Process the book text
    book_doc = nlp(book_text)

    # Extract sentences and entities
    sent_entity_df = []
    for sent in book_doc.sents:
        entity_list = [ent.text for ent in sent.ents]
        # Normalize entities using the alt_name_map
        normalized_entities = [alt_name_map.get(ent, ent) for ent in entity_list]
        sent_entity_df.append({"sentence": sent, "entities": normalized_entities})

    sent_entity_df = pd.DataFrame(sent_entity_df)

    # Filter entities based on the character list
    sent_entity_df['character_entities'] = sent_entity_df['entities'].apply(
        lambda x: [ent for ent in x if ent in list(character_df.character)]
    )
    sent_entity_df_fil = sent_entity_df[sent_entity_df['character_entities'].map(len) > 0]

    # Extract relationships
    relations = []
    for i in range(len(sent_entity_df_fil)):
        end_i = min(i + window_size, len(sent_entity_df_fil) - 1)
        char_list = [
            item for sublist in sent_entity_df_fil.loc[i:end_i, 'character_entities'].tolist() for item in sublist
        ]
        char_unq = [char_list[i] for i in range(len(char_list)) if (i == 0) or char_list[i] != char_list[i - 1]]
        if len(char_unq) > 1:
            for idx, a in enumerate(char_unq[:-1]):
                for b in char_unq[idx + 1:]:
                    if a != b:  # Remove self-loops
                        relations.append({'Source': a, 'Target': b})

    # Create a DataFrame for relationships
    relations_df = pd.DataFrame(relations)
    if not relations_df.empty:
        relations_df["value"] = 1
        relations_df = relations_df.groupby(["Source", "Target"], sort=False, as_index=False).sum()
    return relations_df

# Function to create and save the graph
def create_and_save_graph(relations_df, book_number):
    # Create the directed graph
    G = nx.from_pandas_edgelist(
        relations_df, source="Source", target="Target", edge_attr="value", create_using=nx.DiGraph()
    )

    # Create a dynamic graph for the chapter (book)
    net = Network(notebook=True, width="1000px", height="1000px", directed=True)
    node_degree = dict(G.degree)
    nx.set_node_attributes(G, node_degree, 'size')
    net.from_nx(G)

    # Save as GEXF file
    nx.write_gexf(G, f"graph_book_{book_number}.gexf")
    print(f"GEXF file saved for Book {book_number} as graph_book_{book_number}.gexf")

# Main processing function for all books
def process_all_books(folder_path, character_csv, window_size=10):
    # Read the character data and create a mapping for alternative names
    character_df = pd.read_csv(character_csv)
    alt_name_map = {}

    # Populate the alternative name map
    for _, row in character_df.iterrows():
        primary_name = row['character']
        if pd.notna(row['alternative_names']):
            alternatives = [alt.strip() for alt in row['alternative_names'].split(',')]
            for alt in alternatives:
                alt_name_map[alt] = primary_name

    all_books = [b for b in os.scandir(folder_path) if b.is_file() and b.name.endswith(".txt")]

    for idx, book in enumerate(all_books, start=1):
        print(f"Processing Book {idx}: {book.name}")
        relations_df = process_book(book.path, character_df, alt_name_map, window_size)
        if not relations_df.empty:
            create_and_save_graph(relations_df, idx)
        else:
            print(f"No relationships found in Book {idx}: {book.name}")

# Run the process for all books in the folder
books_folder = "/content/drive/MyDrive/mahatxt"
character_csv = "/content/drive/MyDrive/character_files/processed_names_from_books.csv"
process_all_books(books_folder, character_csv)
